In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import sklearn
import warnings
warnings.filterwarnings('ignore')


In [2]:
training = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
training = training.iloc[:, 1:] # Dropping 'Unnamed' column
test = test.iloc[:, 1:] # Dropping 'Unnamed' column

## LDA - Checks on the assumptions
### Ricordarsi la correzione per test multipli

In [3]:
import scipy.stats as stats, pingouin as pg

kingdoms = np.unique(training.loc[:, "Kingdom"])

# Shapiro-test for each feature in each Kingdom
for col in range(5, 69):
    for kingdom in kingdoms:
        stat, p = stats.shapiro(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col])
        print(f'Shapiro-Wilk Test for [{training.columns.tolist()[col]}] in {kingdom} : statistic={stat}, p-value={p}')

# Kolmogorov Smirnov test for each feature in each kingdom
for col in range(5, 69):
    for kingdom in kingdoms:
        mean = np.mean(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col])
        std = np.std(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col])
        cdf = lambda x: stats.norm.cdf(x, loc=mean, scale=std)
        ks_statistic, p_value = stats.kstest(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col], cdf)
        print(f"{training.columns.tolist()[col]} in {kingdom}  P-value {p_value}")
        
        
# Multivariate Normality test considering all features for each Kingdom
for kingdom in np.unique(training.loc[:, "Kingdom"]):
    hz_test = pg.multivariate_normality(training[training.loc[:, 'Kingdom']==kingdom].iloc[:, 5:])
    print(f" {kingdom} ", hz_test)

Shapiro-Wilk Test for [UUU] in arc : statistic=0.9666157644196436, p-value=0.011637842124512741
Shapiro-Wilk Test for [UUU] in bct : statistic=0.9405073064885423, p-value=1.1675070215460055e-29
Shapiro-Wilk Test for [UUU] in inv : statistic=0.7602696525025234, p-value=7.137077998328769e-37
Shapiro-Wilk Test for [UUU] in mam : statistic=0.9476007491043563, p-value=1.1847296418020915e-11
Shapiro-Wilk Test for [UUU] in phg : statistic=0.9714185324411057, p-value=0.0011012917384280946
Shapiro-Wilk Test for [UUU] in plm : statistic=0.9432675402721048, p-value=0.461784827363552
Shapiro-Wilk Test for [UUU] in pln : statistic=0.9089288547803469, p-value=3.882242987339003e-33
Shapiro-Wilk Test for [UUU] in pri : statistic=0.9517595425267541, p-value=6.579343054767416e-05
Shapiro-Wilk Test for [UUU] in rod : statistic=0.9533226688544535, p-value=1.7949816037112456e-05
Shapiro-Wilk Test for [UUU] in vrl : statistic=0.9739065707653801, p-value=6.565075528796975e-20
Shapiro-Wilk Test for [UUU] in v

Shapiro-Wilk Test for [GUC] in pri : statistic=0.9494492657084772, p-value=4.2597754055895455e-05
Shapiro-Wilk Test for [GUC] in rod : statistic=0.9848528431006259, p-value=0.0590538288976287
Shapiro-Wilk Test for [GUC] in vrl : statistic=0.9793942441290907, p-value=1.3840034895521596e-17
Shapiro-Wilk Test for [GUC] in vrt : statistic=0.9757618303570994, p-value=3.847847422006188e-16
Shapiro-Wilk Test for [GUA] in arc : statistic=0.975174731871636, p-value=0.05354729097693696
Shapiro-Wilk Test for [GUA] in bct : statistic=0.9294641223156579, p-value=7.618701211282651e-32
Shapiro-Wilk Test for [GUA] in inv : statistic=0.9594285958768414, p-value=1.0302635762765876e-16
Shapiro-Wilk Test for [GUA] in mam : statistic=0.9501416447791204, p-value=2.6506745042370675e-11
Shapiro-Wilk Test for [GUA] in phg : statistic=0.9801585189423758, p-value=0.012968913309033542
Shapiro-Wilk Test for [GUA] in plm : statistic=0.9100553829231101, p-value=0.15787806759617334
Shapiro-Wilk Test for [GUA] in pln 

Shapiro-Wilk Test for [GGC] in vrl : statistic=0.9248591982123282, p-value=3.0654225906750436e-32
Shapiro-Wilk Test for [GGC] in vrt : statistic=0.968194750502277, p-value=1.1116101023517495e-18
Shapiro-Wilk Test for [GGA] in arc : statistic=0.9828412372286354, p-value=0.2143022819141051
Shapiro-Wilk Test for [GGA] in bct : statistic=0.9063322330335641, p-value=1.1476327818585015e-35
Shapiro-Wilk Test for [GGA] in inv : statistic=0.9012563071904354, p-value=9.261054065731255e-26
Shapiro-Wilk Test for [GGA] in mam : statistic=0.982915471632023, p-value=3.195633141222153e-05
Shapiro-Wilk Test for [GGA] in phg : statistic=0.939522919656051, p-value=8.99569242483656e-07
Shapiro-Wilk Test for [GGA] in plm : statistic=0.9199883588106309, p-value=0.21981036706763657
Shapiro-Wilk Test for [GGA] in pln : statistic=0.9625277579982261, p-value=1.917705167281772e-22
Shapiro-Wilk Test for [GGA] in pri : statistic=0.9897806885609378, p-value=0.37725177268159754
Shapiro-Wilk Test for [GGA] in rod : s

Shapiro-Wilk Test for [UAC] in bct : statistic=0.9650021179180244, p-value=2.2536642231154336e-23
Shapiro-Wilk Test for [UAC] in inv : statistic=0.9661464679976522, p-value=3.958836197742753e-15
Shapiro-Wilk Test for [UAC] in mam : statistic=0.9914480004529683, p-value=0.009616197813267464
Shapiro-Wilk Test for [UAC] in phg : statistic=0.9556814616233145, p-value=2.4102211239776472e-05
Shapiro-Wilk Test for [UAC] in plm : statistic=0.9686159954731985, p-value=0.8576087727074062
Shapiro-Wilk Test for [UAC] in pln : statistic=0.940464577914033, p-value=9.597709790111535e-28
Shapiro-Wilk Test for [UAC] in pri : statistic=0.9863454749352562, p-value=0.16645331424815307
Shapiro-Wilk Test for [UAC] in rod : statistic=0.9849899317417024, p-value=0.06153585584823479
Shapiro-Wilk Test for [UAC] in vrl : statistic=0.9605512745330707, p-value=2.3597685286139325e-24
Shapiro-Wilk Test for [UAC] in vrt : statistic=0.9579667149567985, p-value=1.69792261314472e-21
Shapiro-Wilk Test for [CAA] in arc : 

Shapiro-Wilk Test for [CGG] in bct : statistic=0.8521276583393128, p-value=2.6893931846346275e-42
Shapiro-Wilk Test for [CGG] in inv : statistic=0.8256911920869306, p-value=1.2225396636873997e-32
Shapiro-Wilk Test for [CGG] in mam : statistic=0.6235674158139703, p-value=2.0326937707537286e-30
Shapiro-Wilk Test for [CGG] in phg : statistic=0.8565093038520332, p-value=7.401871665398629e-12
Shapiro-Wilk Test for [CGG] in plm : statistic=0.9606062491003574, p-value=0.7330720148748056
Shapiro-Wilk Test for [CGG] in pln : statistic=0.9128174556270386, p-value=1.4654174342450165e-32
Shapiro-Wilk Test for [CGG] in pri : statistic=0.8101710354933112, p-value=2.218308603522033e-12
Shapiro-Wilk Test for [CGG] in rod : statistic=0.7525085153492804, p-value=1.0022394035004569e-15
Shapiro-Wilk Test for [CGG] in vrl : statistic=0.8336566622023888, p-value=1.4123119125070083e-43
Shapiro-Wilk Test for [CGG] in vrt : statistic=0.7491274287308689, p-value=1.310799844166174e-44
Shapiro-Wilk Test for [AGA]

UUU in vrt  P-value 3.252134912218105e-19
UUC in arc  P-value 0.31423980986782996
UUC in bct  P-value 4.118379197541894e-13
UUC in inv  P-value 0.24323036734428038
UUC in mam  P-value 0.4541446680314334
UUC in phg  P-value 0.05547943407949607
UUC in plm  P-value 0.36009570489206577
UUC in pln  P-value 4.947929694795118e-07
UUC in pri  P-value 0.7255791990557454
UUC in rod  P-value 0.25526228045391086
UUC in vrl  P-value 0.0031894085914048096
UUC in vrt  P-value 0.014784299991629968
UUA in arc  P-value 0.004467941276031118
UUA in bct  P-value 1.7651542537868635e-67
UUA in inv  P-value 2.3693231843231048e-52
UUA in mam  P-value 0.015103911078763873
UUA in phg  P-value 0.001795674391420074
UUA in plm  P-value 0.5039850379194052
UUA in pln  P-value 3.6756053856085496e-51
UUA in pri  P-value 0.002467203473784144
UUA in rod  P-value 0.023156052403702998
UUA in vrl  P-value 4.834138249964855e-16
UUA in vrt  P-value 2.254322223840093e-13
UUG in arc  P-value 0.3592089405570775
UUG in bct  P-val

CCU in inv  P-value 0.01796970702008941
CCU in mam  P-value 0.8949558435824235
CCU in phg  P-value 0.04138445955978798
CCU in plm  P-value 0.9864986495696082
CCU in pln  P-value 0.22829225022618083
CCU in pri  P-value 0.20762163455467286
CCU in rod  P-value 0.6534660033697763
CCU in vrl  P-value 9.072374162301755e-11
CCU in vrt  P-value 5.071149435523173e-07
CCC in arc  P-value 0.27585313960272473
CCC in bct  P-value 1.1596315641820614e-26
CCC in inv  P-value 4.140256136547589e-07
CCC in mam  P-value 0.37742930045647416
CCC in phg  P-value 0.11496900785159991
CCC in plm  P-value 0.8789459502139179
CCC in pln  P-value 7.146083855545954e-22
CCC in pri  P-value 0.19219613498432342
CCC in rod  P-value 0.6833890912335843
CCC in vrl  P-value 5.445358345319232e-21
CCC in vrt  P-value 0.18538015890172133
CCA in arc  P-value 0.5347906569931944
CCA in bct  P-value 1.3607236496668951e-13
CCA in inv  P-value 9.434980561534491e-06
CCA in mam  P-value 0.6211450664858358
CCA in phg  P-value 0.1811347

UAU in pri  P-value 0.21360877427648417
UAU in rod  P-value 0.20157461799660903
UAU in vrl  P-value 0.003606299228005847
UAU in vrt  P-value 0.0019747775714836816
UAC in arc  P-value 0.05955562273471926
UAC in bct  P-value 1.981732964738563e-05
UAC in inv  P-value 0.023454868096509436
UAC in mam  P-value 0.13137459174919386
UAC in phg  P-value 0.15887726438207683
UAC in plm  P-value 0.6551238616399304
UAC in pln  P-value 4.914536928357479e-09
UAC in pri  P-value 0.47325533923254504
UAC in rod  P-value 0.5494720215361494
UAC in vrl  P-value 6.993227728208717e-13
UAC in vrt  P-value 0.0005276071540982046
CAA in arc  P-value 0.09633647245062449
CAA in bct  P-value 1.3910356458749517e-15
CAA in inv  P-value 4.0062310514892435e-05
CAA in mam  P-value 0.00016295060369101846
CAA in phg  P-value 0.6338240206144411
CAA in plm  P-value 0.9879091699539604
CAA in pln  P-value 6.245375593668561e-74
CAA in pri  P-value 0.07012616934424698
CAA in rod  P-value 0.007159685365861487
CAA in vrl  P-value 

GAA in bct  P-value 0.003251940213539041
GAA in inv  P-value 1.5307288701765396e-08
GAA in mam  P-value 2.530343543488428e-07
GAA in phg  P-value 0.4219370467939765
GAA in plm  P-value 0.833305539624265
GAA in pln  P-value 0.0007166318135267173
GAA in pri  P-value 0.15791704083015234
GAA in rod  P-value 4.828885135311699e-09
GAA in vrl  P-value 9.110496998412734e-05
GAA in vrt  P-value 7.321876700559289e-46
GAG in arc  P-value 0.1304120066482587
GAG in bct  P-value 4.001493404827145e-14
GAG in inv  P-value 2.4132909663532215e-07
GAG in mam  P-value 1.0367049959507899e-44
GAG in phg  P-value 0.04027565852830972
GAG in plm  P-value 0.9833477726880869
GAG in pln  P-value 6.9192397427196e-17
GAG in pri  P-value 1.3097645305292666e-06
GAG in rod  P-value 2.7113026222053324e-16
GAG in vrl  P-value 0.0005826497736771568
GAG in vrt  P-value 8.421292005557147e-112
UAA in arc  P-value 0.10419762012706757
UAA in bct  P-value 1.7286257217253076e-19
UAA in inv  P-value 8.673607361445926e-19
UAA in 

In [6]:
best_pvalues = {}

for kingdom in kingdoms:
    best_pvalues[kingdom] = []
    for col in range(5,69):
        stat, p = stats.shapiro(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col])
        print(f'Shapiro-Wilk Test for [{training.columns.tolist()[col]}] in {kingdom} : statistic={stat}, p-value={p}')
        if p >= 0.05:
            best_pvalues[kingdom].append(p)

print(best_pvalues)

# for col in range(5, 69):
#     for kingdom in kingdoms:
#         stat, p = stats.shapiro(training[training.loc[:, "Kingdom"] == kingdom].iloc[:, col])
#         print(f'Shapiro-Wilk Test for [{training.columns.tolist()[col]}] in {kingdom} : statistic={stat}, p-value={p}')

Shapiro-Wilk Test for [UUU] in arc : statistic=0.9666157644196436, p-value=0.011637842124512741
Shapiro-Wilk Test for [UUC] in arc : statistic=0.9825458143576121, p-value=0.20341893047819753
Shapiro-Wilk Test for [UUA] in arc : statistic=0.8748580905112278, p-value=9.870781885997158e-08
Shapiro-Wilk Test for [UUG] in arc : statistic=0.962449918153852, p-value=0.005705021085955642
Shapiro-Wilk Test for [CUU] in arc : statistic=0.9911593147029734, p-value=0.7518495006052159
Shapiro-Wilk Test for [CUC] in arc : statistic=0.9095870821874384, p-value=3.7276950948550684e-06
Shapiro-Wilk Test for [CUA] in arc : statistic=0.9304579471791474, p-value=4.855254641308113e-05
Shapiro-Wilk Test for [CUG] in arc : statistic=0.9571408512262222, p-value=0.0023757973310070332
Shapiro-Wilk Test for [AUU] in arc : statistic=0.9560974101051283, p-value=0.0020086571050351183
Shapiro-Wilk Test for [AUC] in arc : statistic=0.924940097130569, p-value=2.3791768197624076e-05
Shapiro-Wilk Test for [AUA] in arc : 

Shapiro-Wilk Test for [CCA] in bct : statistic=0.9319931860021677, p-value=2.2772986627778703e-31
Shapiro-Wilk Test for [CCG] in bct : statistic=0.9362468989561081, p-value=1.5466921693770447e-30
Shapiro-Wilk Test for [UGG] in bct : statistic=0.9621295934239981, p-value=2.941384578845449e-24
Shapiro-Wilk Test for [GGU] in bct : statistic=0.9468155230475389, p-value=2.892369994701788e-28
Shapiro-Wilk Test for [GGC] in bct : statistic=0.9450148515810844, p-value=1.1242235944644212e-28
Shapiro-Wilk Test for [GGA] in bct : statistic=0.9063322330335641, p-value=1.1476327818585015e-35
Shapiro-Wilk Test for [GGG] in bct : statistic=0.943689452592087, p-value=5.693163072939109e-29
Shapiro-Wilk Test for [UCU] in bct : statistic=0.9130351443419868, p-value=1.2086546251105006e-34
Shapiro-Wilk Test for [UCC] in bct : statistic=0.957557757511648, p-value=1.4449784981501936e-25
Shapiro-Wilk Test for [UCA] in bct : statistic=0.8878418509738604, p-value=3.266991257312684e-38
Shapiro-Wilk Test for [UCG

Shapiro-Wilk Test for [CAG] in mam : statistic=0.5973490848617349, p-value=3.148462796385279e-31
Shapiro-Wilk Test for [AAU] in mam : statistic=0.9851780890416019, p-value=0.00012723104657273496
Shapiro-Wilk Test for [AAC] in mam : statistic=0.9407341454098401, p-value=1.5177851195164167e-12
Shapiro-Wilk Test for [UGU] in mam : statistic=0.7522627134541889, p-value=1.2301207381957805e-25
Shapiro-Wilk Test for [UGC] in mam : statistic=0.8339315798691813, p-value=1.8097920197361202e-21
Shapiro-Wilk Test for [CAU] in mam : statistic=0.9710538623442417, p-value=7.190560504927999e-08
Shapiro-Wilk Test for [CAC] in mam : statistic=0.9902234563910761, p-value=0.0038907708505437496
Shapiro-Wilk Test for [AAA] in mam : statistic=0.9037849929733486, p-value=2.0387830110520614e-16
Shapiro-Wilk Test for [AAG] in mam : statistic=0.6118216998337898, p-value=8.706407591748311e-31
Shapiro-Wilk Test for [CGU] in mam : statistic=0.8618845874473959, p-value=1.1103349798982152e-19
Shapiro-Wilk Test for [C

Shapiro-Wilk Test for [GCA] in plm : statistic=0.9874298887676219, p-value=0.9979539163372149
Shapiro-Wilk Test for [GCG] in plm : statistic=0.9406322791183424, p-value=0.4264866248113092
Shapiro-Wilk Test for [CCU] in plm : statistic=0.9831404111554656, p-value=0.9890747637054649
Shapiro-Wilk Test for [CCC] in plm : statistic=0.9388011008639492, p-value=0.4031767639430686
Shapiro-Wilk Test for [CCA] in plm : statistic=0.9153208069176552, p-value=0.18822511943900172
Shapiro-Wilk Test for [CCG] in plm : statistic=0.9861129293436776, p-value=0.9962614446935443
Shapiro-Wilk Test for [UGG] in plm : statistic=0.972922025322894, p-value=0.913112956876784
Shapiro-Wilk Test for [GGU] in plm : statistic=0.9224808185807702, p-value=0.23868766255452356
Shapiro-Wilk Test for [GGC] in plm : statistic=0.9436681683432134, p-value=0.46732877618073587
Shapiro-Wilk Test for [GGA] in plm : statistic=0.9199883588106309, p-value=0.21981036706763657
Shapiro-Wilk Test for [GGG] in plm : statistic=0.939842649

Shapiro-Wilk Test for [GAC] in pri : statistic=0.9529249298950656, p-value=8.223145321271686e-05
Shapiro-Wilk Test for [GAA] in pri : statistic=0.9628012012879565, p-value=0.0006072070031002001
Shapiro-Wilk Test for [GAG] in pri : statistic=0.8571624602956378, p-value=1.7024910176397393e-10
Shapiro-Wilk Test for [UAA] in pri : statistic=0.9541394364168115, p-value=0.00010403197021915581
Shapiro-Wilk Test for [UAG] in pri : statistic=0.7756815972200697, p-value=1.436858168799904e-13
Shapiro-Wilk Test for [UGA] in pri : statistic=0.8395279588996452, p-value=3.0256265674246824e-11
Shapiro-Wilk Test for [UUU] in rod : statistic=0.9533226688544535, p-value=1.7949816037112456e-05
Shapiro-Wilk Test for [UUC] in rod : statistic=0.9698487028502017, p-value=0.0008605464886566974
Shapiro-Wilk Test for [UUA] in rod : statistic=0.9576050318704631, p-value=4.563723880248165e-05
Shapiro-Wilk Test for [UUG] in rod : statistic=0.8535845188071195, p-value=7.781773662236141e-12
Shapiro-Wilk Test for [CUU

Shapiro-Wilk Test for [GCC] in vrl : statistic=0.8464914667186387, p-value=2.3783666885632162e-42
Shapiro-Wilk Test for [GCA] in vrl : statistic=0.9937067936258355, p-value=2.8950674333645392e-08
Shapiro-Wilk Test for [GCG] in vrl : statistic=0.7380806905203864, p-value=7.486176163545678e-51
Shapiro-Wilk Test for [CCU] in vrl : statistic=0.9640418850386016, p-value=2.5750901471178734e-23
Shapiro-Wilk Test for [CCC] in vrl : statistic=0.8617778755448391, p-value=8.930544354146386e-41
Shapiro-Wilk Test for [CCA] in vrl : statistic=0.977824484874815, p-value=2.7243340450833485e-18
Shapiro-Wilk Test for [CCG] in vrl : statistic=0.8625435454538135, p-value=1.0799355077393895e-40
Shapiro-Wilk Test for [UGG] in vrl : statistic=0.8824764819997641, p-value=2.1133179496958616e-38
Shapiro-Wilk Test for [GGU] in vrl : statistic=0.9654065789248731, p-value=6.872495946652252e-23
Shapiro-Wilk Test for [GGC] in vrl : statistic=0.9248591982123282, p-value=3.0654225906750436e-32
Shapiro-Wilk Test for [G

In [7]:
print(best_pvalues)

{'arc': [0.20341893047819753, 0.7518495006052159, 0.05354729097693696, 0.1285143912626759, 0.2143022819141051, 0.19124356988652308, 0.12473935811595771, 0.24977311515735656, 0.1042887120723045, 0.05207492023410621], 'bct': [], 'inv': [], 'mam': [0.11999114820401541, 0.17028675125587323, 0.4030020115541409], 'phg': [0.10787899759363556, 0.1253804677244198, 0.5479401608875891], 'plm': [0.461784827363552, 0.06853159346995705, 0.05407603622577594, 0.4894934735977563, 0.3698299784567896, 0.2163524389137806, 0.05903480389102692, 0.6039920634974922, 0.8920359561443367, 0.2037846003984291, 0.2146762010310223, 0.2211706648778924, 0.2474888410405025, 0.64579161966143, 0.15787806759617334, 0.45961263110791195, 0.9309082628528463, 0.4128262474684062, 0.9979539163372149, 0.4264866248113092, 0.9890747637054649, 0.4031767639430686, 0.18822511943900172, 0.9962614446935443, 0.913112956876784, 0.23868766255452356, 0.46732877618073587, 0.21981036706763657, 0.41631147783590905, 0.7009621232952444, 0.97636

## Variance of each feature in each kingdom

In [4]:
for col in range(5, 69):
    feature_name = training.columns.tolist()[col]
    for kingdom in kingdoms:
        training_subset = training[training.loc[:, "Kingdom"]==kingdom].iloc[:, col]
        codon_variance = np.var(training_subset, axis = 0)
        print("Variance of {} in {}: {}".format(feature_name, kingdom, codon_variance))

#np.cov(training.iloc[:, 5:])

Variance of UUU in arc: 9.410804236839525e-05
Variance of UUU in bct: 0.00019007150304933427
Variance of UUU in inv: 0.0002829174563241076
Variance of UUU in mam: 0.00012048901409584104
Variance of UUU in phg: 0.00013916403283832646
Variance of UUU in plm: 4.270145969387755e-05
Variance of UUU in pln: 0.00018740978011896894
Variance of UUU in pri: 0.00011090618238811728
Variance of UUU in rod: 0.00011800639295227148
Variance of UUU in vrl: 9.890309640855909e-05
Variance of UUU in vrt: 0.00019504367678354988
Variance of UUC in arc: 6.0317925497500244e-05
Variance of UUC in bct: 8.99564639071003e-05
Variance of UUC in inv: 0.00011363053210292838
Variance of UUC in mam: 0.00011377549965485022
Variance of UUC in phg: 2.9845581663223144e-05
Variance of UUC in plm: 2.878419591836735e-05
Variance of UUC in pln: 6.852938379738449e-05
Variance of UUC in pri: 9.309429043209877e-05
Variance of UUC in rod: 0.0001690526531199297
Variance of UUC in vrl: 5.462863699981969e-05
Variance of UUC in vrt: 

Variance of GCC in pri: 9.974208726369599e-05
Variance of GCC in rod: 7.187668999459167e-05
Variance of GCC in vrl: 0.00011536558337803898
Variance of GCC in vrt: 0.0001367855872428721
Variance of GCA in arc: 9.72763967258112e-05
Variance of GCA in bct: 9.392215053004493e-05
Variance of GCA in inv: 9.798182166731734e-05
Variance of GCA in mam: 6.231645890238554e-05
Variance of GCA in phg: 5.1316878689953516e-05
Variance of GCA in plm: 2.0482663265306123e-05
Variance of GCA in pln: 6.451702305221784e-05
Variance of GCA in pri: 3.8168738421103395e-05
Variance of GCA in rod: 4.108452321187128e-05
Variance of GCA in vrl: 4.8557297757973574e-05
Variance of GCA in vrt: 8.404336770947824e-05
Variance of GCG in arc: 0.00016835998894226055
Variance of GCG in bct: 0.00033796748556945194
Variance of GCG in inv: 7.217498019677381e-05
Variance of GCG in mam: 8.758087207051737e-06
Variance of GCG in phg: 0.00010722642045131715
Variance of GCG in plm: 0.0001362973775510204
Variance of GCG in pln: 8.2

Variance of UAC in bct: 3.919130590049728e-05
Variance of UAC in inv: 8.688685515332845e-05
Variance of UAC in mam: 3.239600058541981e-05
Variance of UAC in phg: 2.557790877776343e-05
Variance of UAC in plm: 1.0528916836734698e-05
Variance of UAC in pln: 5.876461247386014e-05
Variance of UAC in pri: 2.644769905478395e-05
Variance of UAC in rod: 3.975843335248783e-05
Variance of UAC in vrl: 4.212138537011534e-05
Variance of UAC in vrt: 4.840621524966657e-05
Variance of CAA in arc: 7.790815621997843e-05
Variance of CAA in bct: 0.00015291658733551089
Variance of CAA in inv: 9.556662875815355e-05
Variance of CAA in mam: 1.8094308038080893e-05
Variance of CAA in phg: 6.264021417549071e-05
Variance of CAA in plm: 1.324555510204082e-05
Variance of CAA in pln: 0.00029874764958809754
Variance of CAA in pri: 5.0817504470486114e-05
Variance of CAA in rod: 1.4887353150351544e-05
Variance of CAA in vrl: 5.4182769405357464e-05
Variance of CAA in vrt: 6.688655049839913e-05
Variance of CAG in arc: 4.7

Variance of UGA in rod: 0.0001524455385343429
Variance of UGA in vrl: 1.2031268689774815e-06
Variance of UGA in vrt: 0.0001294293268118602


## LDA - Checks on the outliers

In [11]:
import sklearn

training_with_outliers = training.copy()

for col in range(5, 69):
    considered_variable = training.iloc[:, col].copy()
    Q1 = considered_variable.quantile(0.25)
    Q3 = considered_variable.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    for idx, value in enumerate(considered_variable.tolist()):
        if value < lower or value > upper:
            considered_variable[idx] = True
        else: 
            considered_variable[idx] = False
    training_with_outliers.iloc[:, col] = considered_variable
        


In [12]:
lis = []
for row in range(training_with_outliers.shape[0]):
    lis.append(all(training_with_outliers.iloc[row, 5:]))
sum(lis)

0

## One-hot encoding of DNA type feature

In [13]:
def onehot_feature(pd_data, column_name):
    # Retrieve the unique values (the categories) and an index for each sample
    # specifying the sample category (values[value_idx] reconstruct the original array)
    col_values = pd_data[column_name].to_numpy().astype('<U')
    values, value_idx = np.unique(col_values, return_inverse=True)
    n_values = values.size
    # Create a temporary identity matrix to convert value_idx into one-hot features
    onehots = np.eye(n_values) #when you use an array to index another array in NumPy, it selects rows from the indexed array based on the values in the index array. e[a] selects rows from the identity matrix e based on the values in array a
    value_onehot = onehots[value_idx]
    # Remove the categorical feature
    pd_data = pd_data.drop(column_name, axis=1)
    # Add the new featues
    for i in range(n_values):
        pd_data["{}_{}".format(column_name, values[i])] = value_onehot[:, i]
    return pd_data

In [14]:
training = onehot_feature(training, 'DNAtype')
test = onehot_feature(test, 'DNAtype')

training = training.drop(["SpeciesID", "SpeciesName"], axis = 1)
test = test.drop(["SpeciesID", "SpeciesName"], axis = 1)

## Fitting of the model

We remove the AGA feature from the training set (we will use it at point 4.)

In the test set we add all 0 columns for the DNA type missing (given the fact that a sample with DNA type = 2, is for sure 0 for all others DNA types even though they were never seen in the test set). We have to do this otherwise the function complains.

In [9]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import accuracy_score
import re

# Divide il dataset in feature e target
X = training.drop(["Kingdom", "AGA"], axis = 1)  # Feature
y = training['Kingdom']  # Target

# Adding missing columns for DNA type
pattern = r'DNAtype_\d+'
set_colnames_training = set()
for col in training.columns.tolist():
    if re.match(pattern, col):
        set_colnames_training.add(col)
set_colnames_test = set()
for col in test.columns.tolist():
    if re.match(pattern, col):
        set_colnames_test.add(col)
missing_columns_in_test = list(set_colnames_training.difference(set_colnames_test.intersection(set_colnames_training)))
for col in missing_columns_in_test:
    test[col] = 0

X_test = test.drop(columns = ['Kingdom'])
y_test = test['Kingdom']

X_test = X_test[X.columns]


# Crea un'istanza del modello LDA
model = LDA(store_covariance=True)

# Addestra il modello LDA
model.fit(X, y)

# Valuta il modello
train_accuracy = accuracy_score(y, model.predict(X))

test_accuracy = accuracy_score(y_test, model.predict(X_test))

print("Train accuracy:", train_accuracy)
print("Test accuracy:", test_accuracy)


Train accuracy: 0.8164459796584149
Test accuracy: 0.8069838833461244


In [29]:
len(model.means_) # It's 11 --> 1 mean vector for each kingdom
model.covariance_
model.coef_   
for i in range(len(model.coef_)):  # We have 11 lines, 1 for each kingdom, with their coefficients
    print(len(model.coef_[i]))

75
75
75
75
75
75
75
75
75
75
75


# QDA